# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

# FastSQL

A fun wrapper for SQL.

In [ ]:
#| export
from sqlalchemy import *
from fastcore.utils import *

__all__ = []

In [ ]:
# metadata_obj = MetaData()
# engine = create_engine("sqlite:///:memory:")
# metadata_obj.create_all(engine)

In [ ]:
#| export
class Database:
    def __init__(self, conn_str):
        self.engine = create_engine(conn_str)
        self.metadata = MetaData()
        self.metadata.create_all(self.engine)

In [ ]:
Database("sqlite:///:memory:")

<__main__.Database>

In [ ]:
#| export
type_map = {
    int: Integer,
    str: String,
    float: Float,
    bool: Boolean
}
def create_column(name, typ, primary=False):
    return Column(name, type_map[typ], primary_key=primary)

In [ ]:
create_column("id", int, primary=True)

Column('id', Integer(), table=None, primary_key=True, nullable=False)

In [ ]:
create_column("namme", str)

Column('namme', String(), table=None)

In [ ]:
#| export
@patch
def create_table(self: Database, tname, pk: str|None=None, **cols):
    pkcol = None
    # Set primary key, popping from cols
    if pk is not None: pkcol = create_column(pk, cols.pop(pk), primary=True)
    columns = [create_column(name, typ) for name, typ in cols.items()]
    # Insert primary key at the beginning
    if pkcol is not None: columns.insert(0, pkcol)
    return Table(tname, self.metadata, *columns)

In [ ]:
db = Database("sqlite:///:memory:")
users = db.create_table("users", id=int, name=str, age=int, pk='id')
users
assert 'users' in db.metadata.tables
assert db.metadata.tables['users'] == users

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()